# Accessing the emodels and running custom protocols

This notebook show how to access the optimized emodels and how to run and custom protocols using them.
This notebook does not show how to use the emodel in a circuit or how to export a model in hoc.

## Instantiating an access point to Nexus

In [2]:
species = "mouse"
brain_region = "SSCX"
data_access_point = "nexus"
nexus_project = "ncmv3"
nexus_organisation = "bbp"
nexus_endpoint = "https://bbp.epfl.ch/nexus/v1"
forge_path = "./forge.yml"

emodel = "L5_TPC:B_cAC"
ttype = "L4/5 IT_1"
iteration_tag = "v0"

This next cell will ask for your Nexus token:

In [4]:
from bluepyemodel.access_point.nexus import NexusAccessPoint

access_point = NexusAccessPoint(
    emodel=emodel,
    species=species,
    brain_region=brain_region,
    project=nexus_project,
    organisation=nexus_organisation,
    endpoint=nexus_endpoint,
    forge_path=forge_path,
    ttype=ttype,
    iteration_tag=iteration_tag
)

 ·······································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································

## Retrieving the emodels

In [5]:
emodels = access_point.get_emodels()

In [6]:
import pprint

print(f"Found {len(emodels)} emodels")
print("\nExample of an emodel:\n")
pprint.pprint(emodels[0])

Found 10 emodels

Example of an emodel:

{'brain_region': 'SSCX',
 'emodel': 'L5_TPC:B_cAC',
 'fitness': 294.0428257170971,
 'iteration_tag': 'v0',
 'optimizer': 'SO-CMA',
 'parameters': {'decay_CaDynamics_DC0.axonal': 29.2196061115512,
                'decay_CaDynamics_DC0.somatic': 20.000000139999997,
                'e_pas.all': -86.9625476195746,
                'gCa_HVAbar_Ca_HVA2.apical': 8.035531056383002e-05,
                'gCa_HVAbar_Ca_HVA2.axonal': 2.4999998599062323e-11,
                'gCa_HVAbar_Ca_HVA2.basal': 4.037300775858469e-05,
                'gCa_HVAbar_Ca_HVA2.somatic': 0.023832424074658957,
                'gCa_LVAstbar_Ca_LVAst.apical': 0.0003797820510194191,
                'gCa_LVAstbar_Ca_LVAst.axonal': 0.017038427353158404,
                'gCa_LVAstbar_Ca_LVAst.basal': 0.0007234824814631182,
                'gCa_LVAstbar_Ca_LVAst.somatic': 0.005916668854316305,
                'gIhbar_HCN1.axonal': 3.808573798478965e-06,
                'gIhbar_HCN1.som

## Running protocols and the emodel

Instantiate the evaluator which contains the model (except for the value of the parameters that we got above) and the protocols used for optimization:

In [7]:
from bluepyemodel.evaluation.evaluation import get_evaluator_from_access_point

evaluator = get_evaluator_from_access_point(
    access_point,
    stochasticity=False,
    include_validation_protocols=False,
)

--No graphics will be displayed.


To rerun the protocols that where used for optimization on the first of the models, do:

In [18]:
responses = evaluator.run_protocols(protocols=evaluator.fitness_protocols.values(), param_values=emodels[0].parameters)

If you wish to run additional protocols, you can first define a new protocol:

In [19]:
from bluepyemodel.evaluation.protocol_configuration import ProtocolConfiguration
from bluepyemodel.evaluation.evaluator import define_protocol

protocol_name = "my_step"

stimuli = [{
    "holding_current": -0.08,
    "amp": 0.85,
    "delay": 100,
    "duration": 1000,
    "totduration": 1200
}]
#"thresh_perc": 358.09518664582686,

recordings = [
    {
        "type": "CompRecording",
        "name": f"{protocol_name}.soma.v",
        "location": "soma",
        "variable": "v",
    }
]

my_protocol_configuration = ProtocolConfiguration(
    name=protocol_name,
    stimuli=stimuli,
    recordings=recordings,
    validation=False
)

my_protocol = define_protocol(my_protocol_configuration)

Note that the name of the protocol matters as BluePyEmodel will use it to detect which shape the protocol should have. For example, here, "step" is in the name and therefore a step protocol will be generated. A complete list of the eCode that can be used to create protocols can be seen in bluepyemodel/ecode/__init__.py. For more complex eCode, contact Tanguy for an explanation of how to use them.

Once the protocol is created, it can be added to the protocols already in the evaluator:

In [20]:
evaluator.fitness_protocols["main_protocol"].other_protocols[protocol_name] = my_protocol

In [ ]:
responses = evaluator.run_protocols(protocols=evaluator.fitness_protocols.values(), param_values=emodels[0].parameters)

Finally, the responses can be plotted and saved in a pdf in the current working direcotry:

In [ ]:
from bluepyemodel.emodel_pipeline import plotting

plotting.traces(model=emodels[0], responses=responses, figures_dir="./")